<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Intro-Operations-Research/blob/main/FLINK_Standortplanung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie FLINK Standortplanung

In [1]:
!pip install -U pip
!pip install ortools
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

In [2]:
# Solver mit SCIP als Backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

## Problemdaten laden

Dateien auf Drive hochladen und über Menü auf der Rechtenseite ("Dateien") freigeben!

In [3]:
nachfrage = pd.read_csv('drive/MyDrive/Fallstudie/Nachfrage.csv', sep=';',index_col=0)

In [4]:
nachfrage

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7,91,34,77,33,47,43,51,81,28,88,74,81
1,73,82,76,51,80,42,83,19,37,37,85,18,67
2,85,57,67,14,1,92,41,32,16,83,65,39,18
3,42,87,23,56,15,99,60,85,90,68,9,63,88
4,23,93,36,65,96,31,8,54,27,74,59,70,36
5,2,71,34,38,62,45,86,95,93,12,83,59,17
6,31,41,54,1,97,73,5,73,44,93,94,2,56
7,61,44,86,1,69,65,32,95,61,39,7,61,31
8,59,49,1,70,86,26,62,40,96,82,3,20,83
9,10,38,37,77,16,94,34,67,96,64,32,30,63


In [5]:
standorte = pd.read_csv('drive/MyDrive/Fallstudie//Standorte.csv', sep=';',index_col=0)

In [6]:
standorte

,i_Koordinate,j_Koordinate,Kapazitaet,Kosten
Standorte,,,,
0,1,0,500,100000
1,8,11,600,90000
2,10,4,500,95000
3,10,10,750,101000
4,13,6,725,98500
5,4,8,750,97500
6,13,7,455,105000
7,4,6,3000,225000
8,12,6,1400,300000


## Indexmengen

In [7]:
anzahl_standorte = standorte.shape[0]
anzahl_quadranten_dim_i = nachfrage.shape[0]
anzahl_quadranten_dim_j = nachfrage.shape[1]

In [8]:
K = [s for s in range(standorte.shape[0])] # Menge der potentiellen Standorte für Dark Stores
I = [i for i in range(nachfrage.shape[0])] # Indexmenge der Quadranten in Dimension i
J = [j for j in range(nachfrage.shape[1])] # Indexmenge der Quadranten in Dimension j

In [9]:
# Standorte der Stores
S=standorte.drop(columns=['Kapazitaet','Kosten']).to_numpy()
S

array([[ 1,  0],
       [ 8, 11],
       [10,  4],
       [10, 10],
       [13,  6],
       [ 4,  8],
       [13,  7],
       [ 4,  6],
       [12,  6],
       [ 4,  4]])

## Parameter

In [10]:
# Nachfrage
d = nachfrage.to_numpy()

In [11]:
cap = standorte.Kapazitaet.to_numpy()

In [12]:
c = standorte.Kosten.to_numpy()

In [13]:
# Budget zur Errichtung neuer Dark Stores
B = 1000000

In [14]:
# Maximale Distanz in Anzahl Quadranten
max_lieferzeit = 5

In [15]:
dist={}
for k in K:
    for i in I:
        for j in J:
            dist[k,i,j] = abs(i-S[k][0])+abs(j-S[k][1])

## Entscheidungsvariablen

In [16]:
# Entscheidungsvariablen zum Öffnen von Dark Stores
# y[k]=1 -> Dark Store k öffnen
# y[k]=0 -> Dark Store k nicht öffnen
y={}
for k in K: 
    y[k] = solver.BoolVar(f'{k}')

In [17]:
# Entscheidungsvariablen, die festlegt, welcher Nachfragequadrant von welchem Dark Store beliefert wird
# x[i,j,k]=1 -> Nachfragequadrant (i,j) wird von Dark Store k beliefert
# x[i,j,k]=0 -> Nachfragequadrant (i,j) wird nicht von Dark Store k beliefert
x={}
for k in K:
    for i in I:
        for j in J:
            x[i,j,k] = solver.BoolVar(f'{i,j,k}')

In [18]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 1700


## Zielfunktion

In [19]:
# Maximiere die Anzahl bedienter Bestellungen
solver.Maximize(sum(d[i][j]*x[i,j,k] for k in K for i in I for j in J))

## Nebenbedingungen

In [20]:
# Die Lagerkapazitäten der Stores müssen eingehalten werden
for k in K:
    solver.Add(sum(d[i][j]*x[i,j,k] for i in I for j in J)<=cap[k]*y[k])

In [21]:
# Das zur Verfügung stehende Budget darf nicht überschritten werden
solver.Add(sum(c[k]*y[k] for k in K)<=B)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f7047702f60> >

In [22]:
# Ein Nachfragequadrant darf nur von einem Store bedient werden
for i in I:
    for j in J:
        solver.Add(sum(x[i,j,k] for k in K)<=1)

In [23]:
dist[0,0,0]

1

In [24]:
# Maximale Lieferzeit
for k in K:
    for i in I:
        for j in J:
            solver.Add(dist[k,i,j]*x[i,j,k] <= max_lieferzeit)

In [25]:
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

Anzahl Nebenbedingungen = 1870


## Berechnung Lösung

In [26]:
status = solver.Solve()

In [27]:
if status == pywraplp.Solver.OPTIMAL:
    zielfunktionswert = round(solver.Objective().Value())
    print('LÖSUNG:')
    print('Bediente Nachfrage [Anzahl Bestellungen] =', zielfunktionswert)
    print('Bediente Nachfrage [%] =', zielfunktionswert/np.sum(d))
    invest = 0
    for k in K:
        print(f'{k} =', y[k].solution_value())
        if y[k].solution_value()>0.0:
            invest += y[k].solution_value()*c[k]
    print('Investionen [€] =', invest)
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Bediente Nachfrage [Anzahl Bestellungen] = 7482
Bediente Nachfrage [%] = 0.8702023726448012
0 = 0.0
1 = 1.0
2 = 1.0
3 = 1.0
4 = 1.0
5 = 1.0
6 = 0.0
7 = 1.0
8 = 0.0
9 = 1.0
Investionen [€] = 907000.0
